# Choroid Plexus and Thalamic Nuclei

## Setup

Formatting

<style>
body, .jp-Notebook {
    font-family: 'Source Sans 3', serif !important;
}


table {
    font-family: 'Source Sans 3', sans-serif;
}

table td, table th {
    text-align: center !important;
}

/* Optional: keep first column left-aligned if it contains labels */
table td:first-child, table th:first-child {
    text-align: left !important;
}
</style>

### Imports

In [1]:
from warnings import simplefilter

import pandas as pd

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
simplefilter(action="ignore", category=DeprecationWarning)
simplefilter(action="ignore", category=FutureWarning)

import re
import textwrap
from pathlib import Path
from pyprocessmacro import Process
from statsmodels.stats.multitest import multipletests
from IPython.display import clear_output

import os
import numpy as np
import pyperclip
import statsmodels.api as sm
from matplotlib import colormaps
from scipy import stats
from IPython.display import Markdown, display
from io import StringIO
import matplotlib.pyplot as plt
from collections import defaultdict
import subprocess

from reload_recursive import reload_recursive
import sys

sys.path.insert(0, "/home/srs-9/Projects/ms_mri/analysis/thalamus/helpers")

import helpers
import utils
import regression_utils as regutils
import my_namespace

In [2]:
reload_recursive(regutils)
reload_recursive(helpers)
reload_recursive(utils)
reload_recursive(my_namespace)

import utils
from utils import load_df, zscore, get_colors
from my_namespace import *

import regression_utils as regutils
from regression_utils import (
    quick_regression,
    quick_regression2,
    residualize_vars,
    run_regressions,
)

### Load Data

#### Clinical and Volumes

In [ ]:
fig_path = Path(
    "/home/srs-9/Projects/ms_mri/analysis/thalamus/results/figures_tables/6-Thalamic_nuclei_and_disability"
)
if not os.path.exists(fig_path):
    os.makedirs(fig_path)

data = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/analysis/thalamus/results/data.csv", index_col="subid"
)
dataT = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/analysis/thalamus/results/data_transformed.csv",
    index_col="subid",
)

MS_patients = data["dz_type2"] == "MS"
NONMS_patients = data["dz_type2"] == "!MS"
NIND_patients = data["dz_type5"] == "NIND"
OIND_patients = data["dz_type5"] == "OIND"
RMS_patients = data["dz_type5"] == "RMS"
PMS_patients = data["dz_type5"] == "PMS"

thalamic_nuclei = [2, 4, 5, 6, 7, 8, 9, 10, 11, 12]
deep_grey = [13, 14, 26, 27, 28, 29, 30, 31, 32]
thalamic_nuclei_str = [str(i) for i in thalamic_nuclei]

hips_thomas_ref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="index"
)["struct"]
hips_thomas_invref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="struct"
)["index"]

data_ms = data[MS_patients]
data_ms_z = zscore(data_ms)

resid_data_z = data_ms_z.copy()
resid_data = data_ms.copy()
for var in all_predictors:
    resid_data[var] = utils.residualize_structs(data_ms, var, ["age", "Female", "tiv"])
    resid_data_z[var] = utils.residualize_structs(
        data_ms_z, var, ["age", "Female", "tiv"]
    )


grouped_nuclei = ["medial", "posterior", "ventral", "anterior"]

nuclei_rename_index = {k: f"**{k}**" for k in grouped_nuclei}

### Functions

In [ ]:
def run_R_script(p1, p2, p12, nobs):
    """
    Call R script to run Williams-Hotelling test and return p-value.

    Parameters:
    -----------
    p1 : float
        First correlation coefficient
    p2 : float
        Second correlation coefficient
    p12 : float
        Correlation between the two variables
    nobs : int
        Number of observations

    Returns:
    --------
    float
        The p-value from the Williams-Hotelling test
    """
    # Path to the R script (adjust this to where your R script is located)
    r_script_path = (
        "/home/srs-9/Projects/ms_mri/analysis/thalamus/run_williams_hotelling.R"
    )

    # Convert arguments to strings
    args = [str(p1), str(p2), str(p12), str(nobs)]

    # Call R script
    try:
        result = subprocess.run(
            ["Rscript", r_script_path] + args,
            capture_output=True,
            text=True,
            check=True,
        )

        # Parse the output (p-value)
        p_value = float(result.stdout.strip())
        return p_value

    except subprocess.CalledProcessError as e:
        print(f"Error running R script: {e}")
        print(f"stdout: {e.stdout}")
        print(f"stderr: {e.stderr}")
        raise
    except ValueError as e:
        print(f"Error parsing R output: {e}")
        print(f"R output was: {result.stdout}")
        raise


def thalamus_influence_analysis(model_data, influenced_var, covariates):
    residualized_vars = {}
    for var in ["medial", "posterior", "anterior", "ventral", "THALAMUS_1"] + [
        influenced_var
    ]:
        residualized_vars[var] = residualize_vars(
            model_data,
            dependent_var=var,
            independent_vars=covariates,
        )

    residualized_vars = pd.DataFrame(residualized_vars).dropna()

    nobs = residualized_vars[influenced_var].shape[0]

    structs = ["medial", "posterior", "anterior", "ventral"]
    working_structs = structs.copy()

    nuclei_comps = {}
    thalamus_comps = {}
    pearson_results = {}
    pearson_results["THALAMUS_1"] = stats.pearsonr(
        residualized_vars["THALAMUS_1"], residualized_vars[influenced_var]
    )
    for i, struct in enumerate(structs):
        pearson_results[struct] = stats.pearsonr(
            residualized_vars[struct], residualized_vars[influenced_var]
        )

        thalamus_comps[f"{struct} vs THALAMUS_1"] = run_R_script(
            pearson_results["THALAMUS_1"][0],
            pearson_results[struct][0],
            stats.pearsonr(residualized_vars["THALAMUS_1"], residualized_vars[struct])[
                0
            ],
            nobs,
        )
        if thalamus_comps[f"{struct} vs THALAMUS_1"] >= 0.05:
            sign = "="
        elif abs(pearson_results[struct][0]) > abs(pearson_results["THALAMUS_1"][0]):
            sign = ">"
        else:
            sign = "<"
        thalamus_comps = {re.sub(r"vs", sign, k): v for k, v in thalamus_comps.items()}

    for i, struct1 in enumerate(structs):
        working_structs = working_structs[1:]
        for struct2 in working_structs:
            nuclei_comps[f"{struct1} vs {struct2}"] = run_R_script(
                pearson_results[struct1][0],
                pearson_results[struct2][0],
                stats.pearsonr(residualized_vars[struct1], residualized_vars[struct2])[
                    0
                ],
                nobs,
            )
            if nuclei_comps[f"{struct1} vs {struct2}"] >= 0.05:
                sign = "="
            elif abs(pearson_results[struct1][0]) > abs(pearson_results[struct2][0]):
                sign = ">"
            else:
                sign = "<"
            nuclei_comps = {re.sub(r"vs", sign, k): v for k, v in nuclei_comps.items()}

    return pearson_results, thalamus_comps, nuclei_comps

## Results section 6

**No relationship between clinical disability and regional thalamic atrophy**

In [ ]:
model_data = data_ms_z
save_path = fig_path / "combined_nuclei_thalamus_control_by_struct.xlsx"
covariates = ["age", "Female", "tiv"]

outcomes = ["EDSS_sqrt", "SDMT"]

predictors = [
    "CP",
    "brain",
    "white",
    "grey",
    "THALAMUS_1",
    "LV_log",
    "thirdV_log",
    "fourthV_log",
    "periCSF",
    "allCSF",
]

results, _, models = regutils.run_regressions(
    model_data,
    outcomes,
    predictors,
    covariates,
)


for outcome in outcomes:
    results_to_display = results[outcome]
    display_order = (
        results_to_display["coef"].apply(np.abs).sort_values(ascending=False).index
    )
    display(Markdown(f"`{outcome} ~ *predictor* + {' + '.join(covariates)}`"))
    display(
        Markdown(
            regutils.present_model(results_to_display, presentation_cols).to_markdown()
        )
    )

### How do thalamic nuclei predict disability?


No thalamic nucleus predicts disability after controlling for global thalamic atrophy

In [ ]:
model_data = data_ms_z
save_path = fig_path / "combined_nuclei_thalamus_control_by_struct.xlsx"
covariates = ["THALAMUS_1", "age", "Female", "tiv"]

outcomes = ["EDSS_sqrt", "SDMT"]

predictors = ["medial", "posterior", "ventral", "anterior"]

results, _, models = regutils.run_regressions(
    model_data,
    outcomes,
    predictors,
    covariates,
)


for outcome in outcomes:
    results_to_display = results[outcome]
    display_order = (
        results_to_display["coef"].apply(np.abs).sort_values(ascending=False).index
    )
    display(Markdown(f"`{outcome} ~ *predictor* + {' + '.join(covariates)}`"))
    display(
        Markdown(
            regutils.present_model(results_to_display, presentation_cols).to_markdown()
        )
    )

`EDSS_sqrt ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |    coef |   pval |   p_fdr |     se | ci               |   R2 |
|:------------|--------:|-------:|--------:|-------:|:-----------------|-----:|
| medial      |  0.1327 |   0.26 |    0.53 | 0.1185 | [-0.1, 0.366]    | 0.28 |
| posterior   | -0.0877 |   0.51 |    0.64 | 0.1341 | [-0.351, 0.176]  | 0.27 |
| ventral     | -0.0501 |   0.64 |    0.64 | 0.1072 | [-0.261, 0.161]  | 0.27 |
| anterior    | -0.0996 |   0.13 |    0.53 | 0.0659 | [-0.229, 0.0299] | 0.28 |

`SDMT ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |   coef |   pval |   p_fdr |     se | ci               |   R2 |
|:------------|-------:|-------:|--------:|-------:|:-----------------|-----:|
| medial      | 0.0318 |   0.82 |    0.82 | 0.136  | [-0.236, 0.299]  | 0.25 |
| posterior   | 0.0713 |   0.57 |    0.81 | 0.126  | [-0.176, 0.319]  | 0.25 |
| ventral     | 0.0538 |   0.61 |    0.81 | 0.1054 | [-0.153, 0.261]  | 0.25 |
| anterior    | 0.0676 |   0.26 |    0.81 | 0.0596 | [-0.0495, 0.185] | 0.25 |

### LV expansion and thalamic volumes

Does thalamic volume loss have regional relationships with expansion of the central CSF compartment?

**Steps**

1. Residualize out the effect of age, sex, TIV, and peripheral CSF volume from LV and each of the thalamic nuclei
2. Compute the Pearson correlations (r) between the residualized CP and each of the nuclei as well as the whole thalamus
3. Use the [Williams-Hotelling test](https://rdrr.io/github/bcdudek/bcdstats/man/test2r.t2.html) to compare the sizes of the r's (this procedure was used in Bergsland et al. 2020 as well)

In [ ]:
disease_group = "MS"
influenced_var = "EDSS_sqrt"
model_data = data.copy()[(MS_patients)]
model_data = zscore(model_data)

covariates = ["age", "Female", "tiv"]
results = thalamus_influence_analysis(model_data, influenced_var, covariates)

In [ ]:
pearson_results, thalamus_comps, nuclei_comps = results
display(Markdown(f"##### Patients with {disease_group}"))

display(
    Markdown(
        f"**Pearson correlations of residualized thalamic nuclei and {influenced_var}:**"
    )
)
out_str = "```visualprolog\n"
for struct in pearson_results:
    out_str += f"{struct}: r={pearson_results[struct][0]:0.3}, p={pearson_results[struct][1]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


#! try to figure out why the diff option doesnt render with nbconvert even tho it works within vscode
display(Markdown("\n**Comparisons between thalamic nuclei and whole thalamus:**"))
out_str = "```diff\n"
for comp in thalamus_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={thalamus_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={thalamus_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


display(Markdown("\n**Comparisons between thalamic nuclei:**"))
out_str = "```diff\n"
for comp in nuclei_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={nuclei_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={nuclei_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))

##### Patients with MS

**Pearson correlations of residualized thalamic nuclei and LV_log:**

```visualprolog
THALAMUS_1: r=-0.64, p=3.78e-55
medial: r=-0.682, p=2.70e-65
posterior: r=-0.626, p=4.27e-52
anterior: r=-0.38, p=1.86e-17
ventral: r=-0.432, p=1.30e-22
```



**Comparisons between thalamic nuclei and whole thalamus:**

```diff
+ medial > THALAMUS_1: p=1.75e-03
  posterior = THALAMUS_1: p=2.63e-01
- anterior < THALAMUS_1: p=5.93e-17
- ventral < THALAMUS_1: p=3.70e-31
```



**Comparisons between thalamic nuclei:**

```diff
+ medial > posterior: p=1.82e-03
+ medial > anterior: p=2.71e-20
+ medial > ventral: p=6.75e-22
+ posterior > anterior: p=9.69e-14
+ posterior > ventral: p=1.01e-11
  anterior = ventral: p=1.65e-01
```


#### Test the same in all the disease groups

---

In [ ]:
group_data = {
    "all": data.copy(),
    "!MS": data.copy()[(NONMS_patients)],
    "RMS": data.copy()[(RMS_patients)],
    "PMS": data.copy()[(PMS_patients)],
    "NIND": data.copy()[(NIND_patients)],
    "OIND": data.copy()[(OIND_patients)],
}

influenced_var = "LV_log"
covariates = ["periCSF", "age", "Female", "tiv"]

group_results = {}
for disease_group, model_data in group_data.items():
    print(disease_group)
    group_results[disease_group] = thalamus_influence_analysis(
        model_data, influenced_var, covariates
    )

clear_output(wait=False)

In [ ]:
for disease_group in group_data.keys():
    pearson_results, thalamus_comps, nuclei_comps = group_results[disease_group]
    display(Markdown(f"##### {disease_group.upper()} Patients"))

    display(
        Markdown(
            f"**Pearson correlations of residualized thalamic nuclei and {influenced_var}:**"
        )
    )
    out_str = "```visualprolog\n"
    for struct in pearson_results:
        out_str += f"{struct}: r={pearson_results[struct][0]:0.3}, p={pearson_results[struct][1]:0.2e}\n"
    out_str += "```\n"
    display(Markdown(out_str))

    #! try to figure out why the diff option doesnt render with nbconvert even tho it works within vscode
    display(Markdown("\n**Comparisons between thalamic nuclei and whole thalamus:**"))
    out_str = "```diff\n"
    for comp in thalamus_comps:
        if re.match(r"\w+ [>] \w+", comp):
            prefix = "+"
        elif re.match(r"\w+ [<] \w+", comp):
            prefix = "-"
        else:
            prefix = " "
        out_str += f"{prefix} {comp}: p={thalamus_comps[comp]:0.2e}\n"
        # out_str += f"{comp}: p={thalamus_comps[comp]:0.2e}\n"
    out_str += "```\n"
    display(Markdown(out_str))

    display(Markdown("\n**Comparisons between thalamic nuclei:**"))
    out_str = "```diff\n"
    for comp in nuclei_comps:
        if re.match(r"\w+ [>] \w+", comp):
            prefix = "+"
        elif re.match(r"\w+ [<] \w+", comp):
            prefix = "-"
        else:
            prefix = " "
        out_str += f"{prefix} {comp}: p={nuclei_comps[comp]:0.2e}\n"
        # out_str += f"{comp}: p={nuclei_comps[comp]:0.2e}\n"
    out_str += "```\n"
    display(Markdown(out_str))

##### ALL Patients

**Pearson correlations of residualized thalamic nuclei and LV_log:**

```visualprolog
THALAMUS_1: r=-0.621, p=2.55e-61
medial: r=-0.662, p=3.00e-72
posterior: r=-0.613, p=1.74e-59
anterior: r=-0.359, p=1.34e-18
ventral: r=-0.398, p=7.79e-23
```



**Comparisons between thalamic nuclei and whole thalamus:**

```diff
+ medial > THALAMUS_1: p=1.08e-03
  posterior = THALAMUS_1: p=5.28e-01
- anterior < THALAMUS_1: p=5.52e-19
- ventral < THALAMUS_1: p=6.96e-39
```


...

```visualprolog
THALAMUS_1: r=-0.497, p=9.44e-04
medial: r=-0.538, p=2.84e-04
posterior: r=-0.584, p=6.17e-05
anterior: r=-0.276, p=8.06e-02
ventral: r=-0.22, p=1.66e-01
```



**Comparisons between thalamic nuclei and whole thalamus:**

```diff
  medial = THALAMUS_1: p=3.98e-01
  posterior = THALAMUS_1: p=8.77e-02
  anterior = THALAMUS_1: p=1.18e-01
- ventral < THALAMUS_1: p=5.20e-05
```



**Comparisons between thalamic nuclei:**

```diff
  medial = posterior: p=4.94e-01
  medial = anterior: p=7.84e-02
+ medial > ventral: p=1.61e-03
+ posterior > anterior: p=3.87e-02
+ posterior > ventral: p=9.71e-04
  anterior = ventral: p=7.04e-01
```


#### Check influence of CP

In [19]:
disease_group = "MS"
influenced_var = "CP"
model_data = data.copy()[(MS_patients)]
model_data = zscore(model_data)

covariates = ["periCSF", "age", "Female", "tiv"]
results = thalamus_influence_analysis(model_data, influenced_var, covariates)

In [ ]:
pearson_results, thalamus_comps, nuclei_comps = results
display(Markdown(f"##### Patients with {disease_group}"))

display(
    Markdown(
        f"**Pearson correlations of residualized thalamic nuclei and {influenced_var}:**"
    )
)
out_str = "```visualprolog\n"
for struct in pearson_results:
    out_str += f"{struct}: r={pearson_results[struct][0]:0.3}, p={pearson_results[struct][1]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


#! try to figure out why the diff option doesnt render with nbconvert even tho it works within vscode
display(Markdown("\n**Comparisons between thalamic nuclei and whole thalamus:**"))
out_str = "```diff\n"
for comp in thalamus_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={thalamus_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={thalamus_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


display(Markdown("\n**Comparisons between thalamic nuclei:**"))
out_str = "```diff\n"
for comp in nuclei_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={nuclei_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={nuclei_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))

##### Patients with MS

**Pearson correlations of residualized thalamic nuclei and CP:**

```visualprolog
THALAMUS_1: r=-0.418, p=3.83e-21
medial: r=-0.451, p=7.97e-25
posterior: r=-0.433, p=1.03e-22
anterior: r=-0.242, p=1.21e-07
ventral: r=-0.237, p=2.15e-07
```



**Comparisons between thalamic nuclei and whole thalamus:**

```diff
+ medial > THALAMUS_1: p=4.18e-02
  posterior = THALAMUS_1: p=3.23e-01
- anterior < THALAMUS_1: p=8.50e-07
- ventral < THALAMUS_1: p=5.73e-18
```



**Comparisons between thalamic nuclei:**

```diff
  medial = posterior: p=3.94e-01
+ medial > anterior: p=5.14e-08
+ medial > ventral: p=3.85e-12
+ posterior > anterior: p=3.26e-07
+ posterior > ventral: p=1.75e-09
  anterior = ventral: p=9.07e-01
```


### MS Status

The medial nuclei did not predict MS status after controlling for global thalamic atrophy, suggesting that the medial nuclei atrophy at the same rate as the whole structure. MS is associated with relative preservation of ventral nuclei (positive β), and disproportionate atrophy of the posterior and anterior nuclei. 

In [ ]:
model_data = data.copy()[(MS_patients) | (NONMS_patients)]
model_data = zscore(model_data)
model_data = model_data.join([pd.get_dummies(model_data["dz_type2"], dtype=int)])

outcome = "MS"  # LV / allCSF
covariates = ["THALAMUS_1", "CP", "age", "Female", "tiv"]

save_cols = ["coef", "ci", "p_fdr"]

predictors = {
    "medial": ["medial"],
    "posterior": ["posterior"],
    "ventral": ["ventral"],
    "anterior": ["anterior"],
}

results, _, _ = regutils.run_regressions(
    model_data, [outcome], grouped_nuclei, covariates, regression_model=sm.Logit
)

results_to_display = results["MS"]
display(Markdown(f"`MS ~ predictor + {' + '.join(covariates)}`"))
display(
    Markdown(
        regutils.present_model(results_to_display, presentation_cols).to_markdown()
    )
)

results, models, formulas = regutils.run_regressions2(
    model_data,
    outcome,
    predictors.values(),
    covariates=covariates,
    model_names=predictors.keys(),
    regression_model=sm.Logit,
)

for model_name, model, formula in zip(
    results.keys(), results.values(), formulas.values()
):
    display(Markdown(f"#### {model_name}"))
    display(Markdown(f"`{formula}`"))
    display(
        Markdown(
            regutils.present_model(
                model, presentation_cols, rename_index=nuclei_rename_index
            ).to_markdown()
        )
    )

`MS ~ predictor + THALAMUS_1 + CP + age + Female + tiv`

| predictor   |    coef |   pval |   p_fdr |     se | ci                | R2   |
|:------------|--------:|-------:|--------:|-------:|:------------------|:-----|
| medial      | -0.018  | 0.96   |  0.96   | 0.3794 | [-0.762, 0.726]   |      |
| posterior   | -1.4295 | 0.0012 |  0.0049 | 0.4418 | [-2.3, -0.564]    |      |
| ventral     |  0.9197 | 0.0075 |  0.015  | 0.3437 | [0.246, 1.59]     |      |
| anterior    | -0.3625 | 0.042  |  0.056  | 0.1784 | [-0.712, -0.0128] |      |

#### medial

`MS ~ medial + THALAMUS_1 + CP + age + Female + tiv`

|            |    coef |    pval |     se | ci               |
|:-----------|--------:|--------:|-------:|:-----------------|
| Intercept  |  1.9175 | 2e-39   | 0.146  | [1.63, 2.2]      |
| **medial** | -0.018  | 0.96    | 0.3794 | [-0.762, 0.726]  |
| THALAMUS_1 | -1.1598 | 0.0054  | 0.4167 | [-1.98, -0.343]  |
| CP         | -0.1097 | 0.47    | 0.1511 | [-0.406, 0.187]  |
| age        | -0.5078 | 0.00047 | 0.1452 | [-0.792, -0.223] |
| Female     | -0.1447 | 0.33    | 0.1497 | [-0.438, 0.149]  |
| tiv        |  0.2024 | 0.25    | 0.1774 | [-0.145, 0.55]   |

...

`MS ~ ventral + THALAMUS_1 + CP + age + Female + tiv`

|             |    coef |    pval |     se | ci               |
|:------------|--------:|--------:|-------:|:-----------------|
| Intercept   |  1.9557 | 6.4e-39 | 0.1499 | [1.66, 2.25]     |
| **ventral** |  0.9197 | 0.0075  | 0.3437 | [0.246, 1.59]    |
| THALAMUS_1  | -2.0051 | 3.2e-07 | 0.3922 | [-2.77, -1.24]   |
| CP          | -0.2491 | 0.11    | 0.1555 | [-0.554, 0.0558] |
| age         | -0.5333 | 0.00025 | 0.1457 | [-0.819, -0.248] |
| Female      | -0.2204 | 0.15    | 0.1537 | [-0.522, 0.0809] |
| tiv         |  0.0153 | 0.94    | 0.1912 | [-0.359, 0.39]   |

#### anterior

`MS ~ anterior + THALAMUS_1 + CP + age + Female + tiv`

|              |    coef |    pval |     se | ci                |
|:-------------|--------:|--------:|-------:|:------------------|
| Intercept    |  1.939  | 4.7e-39 | 0.1483 | [1.65, 2.23]      |
| **anterior** | -0.3625 | 0.042   | 0.1784 | [-0.712, -0.0128] |
| THALAMUS_1   | -0.9181 | 0.0004  | 0.2592 | [-1.43, -0.41]    |
| CP           | -0.0849 | 0.57    | 0.1483 | [-0.376, 0.206]   |
| age          | -0.6081 | 9.3e-05 | 0.1556 | [-0.913, -0.303]  |
| Female       | -0.1606 | 0.29    | 0.1508 | [-0.456, 0.135]   |
| tiv          |  0.1714 | 0.34    | 0.1779 | [-0.177, 0.52]    |

Compare influence of nuclei on predicting MS versus whole thalamus, and to eachother.

Issue here is that logistic regressions don't have residuals, so I had to use regular OLS when residualizing. 

In [ ]:
influenced_var = "MS"
model_data = data.copy()[(MS_patients) | (NONMS_patients)]
model_data = zscore(model_data)
model_data = model_data.join([pd.get_dummies(model_data["dz_type2"], dtype=int)])

covariates = ["age", "Female", "tiv"]
results2 = thalamus_influence_analysis(model_data, influenced_var, covariates)

In [ ]:
pearson_results, thalamus_comps, nuclei_comps = results2
display(Markdown(f"##### How do thalamic volumes predict MS"))

display(
    Markdown(
        f"**Pearson correlations of residualized thalamic nuclei and {influenced_var}:**"
    )
)
out_str = "```visualprolog\n"
for struct in pearson_results:
    out_str += f"{struct}: r={pearson_results[struct][0]:0.3}, p={pearson_results[struct][1]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


#! try to figure out why the diff option doesnt render with nbconvert even tho it works within vscode
display(Markdown("\n**Comparisons between thalamic nuclei and whole thalamus:**"))
out_str = "```diff\n"
for comp in thalamus_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={thalamus_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={thalamus_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))


display(Markdown("\n**Comparisons between thalamic nuclei:**"))
out_str = "```diff\n"
for comp in nuclei_comps:
    if re.match(r"\w+ [>] \w+", comp):
        prefix = "+"
    elif re.match(r"\w+ [<] \w+", comp):
        prefix = "-"
    else:
        prefix = " "
    out_str += f"{prefix} {comp}: p={nuclei_comps[comp]:0.2e}\n"
    # out_str += f"{comp}: p={nuclei_comps[comp]:0.2e}\n"
out_str += "```\n"
display(Markdown(out_str))

##### How do thalamic volumes predict MS

**Pearson correlations of residualized thalamic nuclei and MS:**

```visualprolog
THALAMUS_1: r=-0.225, p=7.11e-08
medial: r=-0.209, p=6.32e-07
posterior: r=-0.257, p=6.70e-10
anterior: r=-0.215, p=2.87e-07
ventral: r=-0.151, p=3.21e-04
```



**Comparisons between thalamic nuclei and whole thalamus:**

```diff
  medial = THALAMUS_1: p=2.95e-01
+ posterior > THALAMUS_1: p=2.99e-02
  anterior = THALAMUS_1: p=7.60e-01
- ventral < THALAMUS_1: p=2.91e-04
```



**Comparisons between thalamic nuclei:**

```diff
- medial < posterior: p=2.33e-02
  medial = anterior: p=8.71e-01
  medial = ventral: p=5.56e-02
  posterior = anterior: p=2.39e-01
+ posterior > ventral: p=7.45e-04
  anterior = ventral: p=8.64e-02
```


---

#### Overall thalamus size versus disease phenotype

Thalamus is smaller in MS patients

In [ ]:
model_data = data.copy()[(MS_patients) | (NONMS_patients)]
model_data = zscore(model_data)
model_data = model_data.join([pd.get_dummies(model_data["dz_type2"], dtype=int)])

res = sm.Logit.from_formula(
    "MS ~ THALAMUS_1 + age + Female + tiv", data=model_data
).fit(disp=0)
print(res.summary())

                           Logit Regression Results                           
Dep. Variable:                     MS   No. Observations:                  560
Model:                          Logit   Df Residuals:                      555
Method:                           MLE   Df Model:                            4
Date:                Tue, 04 Nov 2025   Pseudo R-squ.:                 0.09205
Time:                        18:07:56   Log-Likelihood:                -227.13
converged:                       True   LL-Null:                       -250.16
Covariance Type:            nonrobust   LLR p-value:                 2.397e-09
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.9026      0.145     13.149      0.000       1.619       2.186
THALAMUS_1    -1.1011      0.208     -5.305      0.000      -1.508      -0.694
age           -0.5342      0.141     -3.788      0.0

Thalamus is smaller in PMS patients

In [ ]:
model_data = data.copy()[(MS_patients)]
model_data = zscore(model_data)
model_data = model_data.join([pd.get_dummies(model_data["dz_type5"], dtype=int)])

res = sm.Logit.from_formula(
    "PMS ~ THALAMUS_1 + age + Female + tiv", data=model_data
).fit(disp=0)
print(res.summary())

                           Logit Regression Results                           
Dep. Variable:                    PMS   No. Observations:                  468
Model:                          Logit   Df Residuals:                      463
Method:                           MLE   Df Model:                            4
Date:                Tue, 04 Nov 2025   Pseudo R-squ.:                  0.2944
Time:                        18:05:52   Log-Likelihood:                -168.50
converged:                       True   LL-Null:                       -238.83
Covariance Type:            nonrobust   LLR p-value:                 2.056e-29
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.0353      0.185    -11.021      0.000      -2.397      -1.673
THALAMUS_1    -0.7105      0.156     -4.554      0.000      -1.016      -0.405
age            1.2298      0.180      6.832      0.0

## Extras

In [ ]:
model_data = data_ms_z
# model_data = zscore(dataT[MS_patients])
save_path = fig_path / "combined_nuclei_thalamus_control_by_struct.xlsx"
covariates = ["THALAMUS_1", "age", "Female", "tiv"]

outcomes = ["medial", "posterior", "ventral", "anterior"]

predictor = ["CP", "t2lv_log", "PRL", "LV_log", "thirdV_log", "CCR"]
# predictor = ["CP", "t2lv", "PRL", "LV", "thirdV", "CCR"]


results, _, models = regutils.run_regressions(
    model_data,
    outcomes,
    predictor,
    covariates,
)

for outcome_to_display in outcomes:
    results_to_display = results[outcome_to_display]
    display_order = (
        results_to_display["coef"].apply(np.abs).sort_values(ascending=False).index
    )
    display(
        Markdown(f"`{outcome_to_display} ~ *predictor* + {' + '.join(covariates)}`")
    )
    display(
        Markdown(
            regutils.present_model(results_to_display, presentation_cols).to_markdown()
        )
    )

`medial ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |    coef |    pval |   p_fdr |     se | ci                |   R2 |
|:------------|--------:|--------:|--------:|-------:|:------------------|-----:|
| CP          | -0.0758 | 0.00021 | 0.00043 | 0.0203 | [-0.116, -0.0359] | 0.89 |
| t2lv_log    | -0.0191 | 0.33    | 0.4     | 0.0197 | [-0.0579, 0.0197] | 0.89 |
| PRL         |  0.0089 | 0.65    | 0.65    | 0.0198 | [-0.0301, 0.0479] | 0.89 |
| LV_log      | -0.1559 | 1.6e-11 | 9.5e-11 | 0.0226 | [-0.2, -0.112]    | 0.9  |
| thirdV_log  | -0.0639 | 0.0038  | 0.0056  | 0.0219 | [-0.107, -0.0208] | 0.89 |
| CCR         | -0.1242 | 1.7e-07 | 5.2e-07 | 0.0234 | [-0.17, -0.0782]  | 0.9  |

`posterior ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:------------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| CP          | -0.0453 | 0.012   |  0.019  | 0.0179 | [-0.0804, -0.0101] | 0.9  |
| t2lv_log    | -0.0712 | 0.00048 |  0.0025 | 0.0203 | [-0.111, -0.0314]  | 0.91 |
| PRL         | -0.0597 | 0.00085 |  0.0025 | 0.0178 | [-0.0947, -0.0248] | 0.9  |
| LV_log      | -0.0532 | 0.013   |  0.019  | 0.0213 | [-0.0951, -0.0113] | 0.9  |
| thirdV_log  |  0.0173 | 0.37    |  0.37   | 0.0194 | [-0.0208, 0.0555]  | 0.9  |
| CCR         | -0.029  | 0.21    |  0.25   | 0.023  | [-0.0741, 0.0161]  | 0.9  |

`ventral ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |   coef |    pval |   p_fdr |     se | ci                |   R2 |
|:------------|-------:|--------:|--------:|-------:|:------------------|-----:|
| CP          | 0.1292 | 1.3e-10 | 4e-10   | 0.0197 | [0.0906, 0.168]   | 0.87 |
| t2lv_log    | 0.09   | 0.00049 | 0.00059 | 0.0256 | [0.0396, 0.14]    | 0.87 |
| PRL         | 0.0249 | 0.2     | 0.2     | 0.0193 | [-0.0131, 0.0629] | 0.86 |
| LV_log      | 0.1981 | 1.7e-15 | 1e-14   | 0.024  | [0.151, 0.245]    | 0.88 |
| thirdV_log  | 0.1019 | 3.5e-05 | 5.3e-05 | 0.0244 | [0.054, 0.15]     | 0.87 |
| CCR         | 0.1505 | 4.1e-09 | 8.2e-09 | 0.0251 | [0.101, 0.2]      | 0.87 |

`anterior ~ *predictor* + THALAMUS_1 + age + Female + tiv`

| predictor   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:------------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| CP          |  0.037  | 0.4     |  0.4    | 0.0439 | [-0.0493, 0.123]   | 0.56 |
| t2lv_log    | -0.0825 | 0.04    |  0.08   | 0.0401 | [-0.161, -0.00376] | 0.57 |
| PRL         | -0.0623 | 0.033   |  0.08   | 0.0292 | [-0.12, -0.00496]  | 0.56 |
| LV_log      |  0.061  | 0.22    |  0.33   | 0.0496 | [-0.0364, 0.158]   | 0.56 |
| thirdV_log  | -0.1526 | 0.00025 |  0.0015 | 0.0414 | [-0.234, -0.0713]  | 0.57 |
| CCR         |  0.0537 | 0.29    |  0.35   | 0.0509 | [-0.0463, 0.154]   | 0.56 |

In [ ]:
model_data = data_ms_z
save_path = fig_path / "combined_nuclei_thalamus_control.xlsx"
covariates = ["THALAMUS_1", "age", "Female", "tiv"]

outcomes = ["medial", "posterior", "ventral", "anterior"]

predictor = ["CP", "t2lv_log", "LV_log", "thirdV_log", "periCSF"]

_, results, models = regutils.run_regressions(
    model_data,
    outcomes,
    predictor,
    covariates,
)

save_cols = ["coef", "ci", "p_fdr"]
with pd.ExcelWriter(save_path) as writer:
    for predictor in results:
        save_results = results[predictor][save_cols]
        save_results.to_excel(writer, sheet_name=predictor)


for outcome_to_display in ["CP", "LV_log", "t2lv_log"]:
    results_to_display = results[outcome_to_display]
    display_order = (
        results_to_display["coef"].apply(np.abs).sort_values(ascending=False).index
    )
    display(Markdown(f"`outcome ~ *{outcome_to_display}* + {' + '.join(covariates)}`"))
    display(
        Markdown(
            regutils.present_model(results_to_display, presentation_cols).to_markdown()
        )
    )

`outcome ~ *CP* + THALAMUS_1 + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:----------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| medial    | -0.0758 | 0.00021 | 0.00043 | 0.0203 | [-0.116, -0.0359]  | 0.89 |
| posterior | -0.0453 | 0.012   | 0.016   | 0.0179 | [-0.0804, -0.0101] | 0.9  |
| ventral   |  0.1292 | 1.3e-10 | 5.4e-10 | 0.0197 | [0.0906, 0.168]    | 0.87 |
| anterior  |  0.037  | 0.4     | 0.4     | 0.0439 | [-0.0493, 0.123]   | 0.56 |

`outcome ~ *LV_log* + THALAMUS_1 + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:----------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| medial    | -0.1559 | 1.6e-11 | 3.2e-11 | 0.0226 | [-0.2, -0.112]     | 0.9  |
| posterior | -0.0532 | 0.013   | 0.017   | 0.0213 | [-0.0951, -0.0113] | 0.9  |
| ventral   |  0.1981 | 1.7e-15 | 6.8e-15 | 0.024  | [0.151, 0.245]     | 0.88 |
| anterior  |  0.061  | 0.22    | 0.22    | 0.0496 | [-0.0364, 0.158]   | 0.56 |

`outcome ~ *t2lv_log* + THALAMUS_1 + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:----------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| medial    | -0.0191 | 0.33    | 0.33    | 0.0197 | [-0.0579, 0.0197]  | 0.89 |
| posterior | -0.0712 | 0.00048 | 0.00098 | 0.0203 | [-0.111, -0.0314]  | 0.91 |
| ventral   |  0.09   | 0.00049 | 0.00098 | 0.0256 | [0.0396, 0.14]     | 0.87 |
| anterior  | -0.0825 | 0.04    | 0.053   | 0.0401 | [-0.161, -0.00376] | 0.57 |

In [ ]:
model_data = data_ms_z
save_path = fig_path / "combined_nuclei_thalamus_periCSF_control.xlsx"
covariates = ["THALAMUS_1", "periCSF", "age", "Female", "tiv"]

outcomes = ["medial", "posterior", "ventral", "anterior"]

predictor = ["CP", "LV_log", "brain", "white", "grey", "cortical_thickness", "t2lv_log"]

_, results, formulas = regutils.run_regressions(
    model_data,
    outcomes,
    predictor,
    covariates,
)

save_cols = ["coef", "ci", "p_fdr"]
with pd.ExcelWriter(save_path) as writer:
    for predictor in results:
        save_results = results[predictor][save_cols]
        save_results.to_excel(writer, sheet_name=predictor)


for outcome_to_display in ["CP", "LV_log", "t2lv_log"]:
    results_to_display = results[outcome_to_display]
    display_order = (
        results_to_display["coef"].apply(np.abs).sort_values(ascending=False).index
    )
    display(Markdown(f"`struct ~ *{outcome_to_display}* + {' + '.join(covariates)}`"))
    display(
        Markdown(
            regutils.present_model(results_to_display, presentation_cols).to_markdown()
        )
    )

`struct ~ *CP* + THALAMUS_1 + periCSF + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:----------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| medial    | -0.0793 | 0.00015 | 0.00029 | 0.0207 | [-0.12, -0.0386]   | 0.89 |
| posterior | -0.0506 | 0.0047  | 0.0063  | 0.0178 | [-0.0856, -0.0156] | 0.91 |
| ventral   |  0.1367 | 2.4e-11 | 9.7e-11 | 0.02   | [0.0975, 0.176]    | 0.88 |
| anterior  |  0.037  | 0.41    | 0.41    | 0.0446 | [-0.0507, 0.125]   | 0.56 |

`struct ~ *LV_log* + THALAMUS_1 + periCSF + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                  |   R2 |
|:----------|--------:|--------:|--------:|-------:|:--------------------|-----:|
| medial    | -0.1563 | 1.5e-11 | 2.9e-11 | 0.0226 | [-0.201, -0.112]    | 0.9  |
| posterior | -0.046  | 0.036   | 0.048   | 0.0218 | [-0.0889, -0.00305] | 0.9  |
| ventral   |  0.1935 | 6e-14   | 2.4e-13 | 0.025  | [0.144, 0.243]      | 0.88 |
| anterior  |  0.0646 | 0.19    | 0.19    | 0.0497 | [-0.0331, 0.162]    | 0.56 |

`struct ~ *t2lv_log* + THALAMUS_1 + periCSF + age + Female + tiv`

| outcome   |    coef |    pval |   p_fdr |     se | ci                 |   R2 |
|:----------|--------:|--------:|--------:|-------:|:-------------------|-----:|
| medial    | -0.0172 | 0.39    |  0.39   | 0.0198 | [-0.0561, 0.0218]  | 0.89 |
| posterior | -0.0673 | 0.00084 |  0.0017 | 0.02   | [-0.107, -0.0279]  | 0.91 |
| ventral   |  0.0854 | 0.0008  |  0.0017 | 0.0253 | [0.0357, 0.135]    | 0.87 |
| anterior  | -0.0827 | 0.039   |  0.052  | 0.04   | [-0.161, -0.00406] | 0.56 |